In [ ]:
from deep_orderbook.replayer import Replayer
from deep_orderbook.shapper import BookShapper
import numpy as np
from pylab import rcParams

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
rcParams['figure.figsize'] = 20, 4

In [ ]:
MARKETS = ["ETHBTC", "BTCUSDT", "ETHUSDT", "BNBBTC", "BNBETH", "BNBUSDT"]

In [ ]:
# itertools for asyncio !
import asyncio
from aioitertools import iter, next, map, zip
from aioitertools import enumerate

In [ ]:
shapper = await BookShapper.create()
file_replayer = Replayer('../crypto-trading/data/L2', date_regexp='2020')
areplay = file_replayer.replayL2_async('ETHBTC', shapper)
await next(areplay)
batptr = await next(areplay)

In [ ]:
for i in range(100):
    batptr = await next(areplay)
print(f"bids:\n{batptr['bids'].head()}")
print(f"asks:\n{batptr['asks'].head()}")
print(f"prices:\n{batptr['price']}")
print(f"trades:\n{batptr['trades']}")

In [ ]:
replayers = [file_replayer.replayL2_async(pair, await BookShapper.create()) for pair in MARKETS]
multi_replay = file_replayer.multireplayL2_async(replayers)
d = await next(multi_replay)

In [ ]:
async def run(maxpoints=100):
    async for i in iter(range(maxpoints)):
        d = await next(multi_replay)
    #return d

task = asyncio.create_task(run())

In [ ]:
await task

In [ ]:
genarr = shapper.gen_array_async(market_replay=multi_replay, markets=MARKETS)
_ = await next(genarr)

In [ ]:
genacc = shapper.accumulate_array(genarr, markets=MARKETS)
_ = await next(genacc)

In [ ]:
async for toshow in BookShapper.images(accumulated_arrays=genacc, every=10, LENGTH=256):
    plt.figure(figsize=(15,11))
    plt.imshow(toshow, origin="lower");
    clear_output(wait=True)
    plt.show()

In [ ]:
#task = asyncio.create_task(images(accumulated_arrays=genacc))